# Music Genre Classification

## Recurrent Neural Network Implementation

Fabrizio Niro - Jacopo Signò

GTZAN Dataset - Music Genre Classification 

https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification

In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
from torchinfo import summary
import pickle
import numpy as np
from d2l import torch as d2l
from torch.utils import data

In [2]:
with open('Processed_data/RNN/data_train', 'rb') as f:
    data_train = pickle.load(f)

with open('Processed_data/RNN/data_test', 'rb') as f:
    data_test = pickle.load(f)

In [3]:
data_train

[[array([[-2.577475  , -1.6451437 , -1.4188532 , ..., -1.3022156 ,
          -1.3229824 , -1.2635036 ],
         [ 2.6294422 ,  2.6998615 ,  2.4782076 , ...,  2.3572288 ,
           2.6633976 ,  3.01722   ],
         [-0.16664992, -0.76356465, -1.1481934 , ..., -1.4294202 ,
          -1.2735374 , -1.1747715 ],
         ...,
         [-0.2928514 , -0.2037775 ,  0.04849779, ..., -0.10467712,
          -0.06604507, -0.18648091],
         [ 0.128238  ,  0.00518849, -0.14782554, ..., -0.49848467,
          -0.3949254 , -0.30046406],
         [-0.15557528, -0.3215027 , -0.13622503, ..., -0.21165025,
          -0.13176343, -0.3984717 ]], dtype=float32),
  9],
 [array([[-3.2884958 , -2.9816196 , -2.6818721 , ..., -1.7729429 ,
          -1.9378313 , -1.8879622 ],
         [ 2.9173863 ,  3.1773934 ,  3.3592317 , ...,  3.4671652 ,
           3.3409815 ,  3.4747076 ],
         [-0.2901911 , -0.27311477, -0.41380638, ..., -1.6467277 ,
          -1.5018992 , -1.2476095 ],
         ...,
         [ 0.

In [4]:
len(data_train[1][0])

20

In [5]:
print(len(data_train), len(data_train[0]))

8000 2


In [6]:
print(len(data_test), len(data_test[0]))

2000 2


In [7]:
batch_size = 64

train_iter = data.DataLoader(data_train, batch_size, shuffle=True,
                             num_workers = 2)

In [8]:
train_iter

In [9]:
next(iter(train_iter))

[tensor([[[-3.3764e+00, -3.2765e+00, -3.3758e+00,  ..., -3.7724e+00,
           -3.8064e+00, -3.8329e+00],
          [ 2.5742e+00,  2.5661e+00,  2.4069e+00,  ...,  1.6629e+00,
            1.6410e+00,  1.7115e+00],
          [-1.2669e-01, -2.1467e-01, -2.9991e-01,  ..., -6.1141e-01,
           -5.3261e-01, -4.4833e-01],
          ...,
          [ 2.7083e-01,  3.4611e-01,  2.7506e-01,  ...,  4.3851e-01,
            3.8245e-01,  4.0177e-01],
          [ 6.5069e-02,  4.2097e-02,  2.6235e-02,  ...,  3.3347e-01,
            2.9160e-01,  2.5518e-01],
          [ 2.2507e-01,  2.1658e-01,  2.5376e-01,  ...,  1.8727e-01,
            1.7696e-01,  1.1412e-01]],
 
         [[-4.1902e+00, -4.1509e+00, -4.1238e+00,  ..., -4.0653e+00,
           -4.0689e+00, -4.0890e+00],
          [ 1.3740e+00,  1.4841e+00,  1.5519e+00,  ...,  1.3043e+00,
            1.2365e+00,  1.2766e+00],
          [ 9.9143e-02,  5.1786e-02,  1.0604e-02,  ..., -4.9452e-01,
           -5.3796e-01, -5.1492e-01],
          ...,
    

In [10]:
test_iter = data.DataLoader(data_test, batch_size, shuffle=True,
                             num_workers = 2)

In [11]:
test_iter

In [12]:
next(iter(test_iter))

[tensor([[[-3.5880e+00, -3.5487e+00, -3.1722e+00,  ..., -3.7703e+00,
           -3.7735e+00, -3.7819e+00],
          [ 2.2591e+00,  2.1048e+00,  1.4674e+00,  ...,  1.4206e+00,
            1.4193e+00,  1.7811e+00],
          [-4.6768e-01,  5.1484e-01,  1.4730e+00,  ..., -1.5160e-01,
           -2.1421e-01, -3.2129e-01],
          ...,
          [ 1.9491e-01,  4.0764e-01,  6.7086e-01,  ...,  3.0217e-02,
            2.3976e-02,  5.9181e-02],
          [ 2.2375e-01,  4.2309e-01,  4.2059e-01,  ..., -6.5157e-02,
           -5.2064e-02,  9.5924e-03],
          [ 1.9151e-01,  4.8107e-02,  7.7271e-02,  ...,  1.6431e-01,
            1.3457e-01,  1.3625e-01]],
 
         [[-4.1860e+00, -3.5906e+00, -2.2444e+00,  ..., -3.3853e+00,
           -3.6637e+00, -3.6856e+00],
          [ 1.1843e+00,  1.3630e+00,  1.2623e+00,  ...,  2.3899e+00,
            2.3279e+00,  2.2981e+00],
          [ 1.4820e-01, -1.1461e+00, -2.0113e+00,  ...,  1.4204e-01,
           -1.7833e-03, -2.7333e-02],
          ...,
    

In [13]:
next(iter(test_iter))[0].shape

torch.Size([64, 20, 129])

In [14]:
next(iter(test_iter))[0][0].shape

torch.Size([20, 129])

In [15]:
device = d2l.try_gpu()

In [16]:
torch.cuda.is_available()

False

In [78]:
input_size = data_train[0][0].shape[1] # 20

hidden_size = 20 # 20

num_layers = 2

dropout = 0

classes_num = 10

In [83]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.gru = nn.GRU(
            input_size=input_size, hidden_size=hidden_size,
             num_layers=num_layers, bias=False, batch_first=True, dropout=dropout, bidirectional=False)
        
        self.relu = nn.ReLU()
        
        self.fc = nn.Linear(hidden_size, classes_num)

        self.sm = nn.Softmax()


    def forward(self, x, h):

        out, h = self.gru(x, h)

        out = self.fc(self.relu(out[:,-1]))

        out = self.sm(out)

        return out, h


    def init_hidden(self, batch_size):

        weight = next(self.parameters()).data

        hidden = weight.new(self.num_layers, batch_size, self.hidden_size).zero_().to(device)

        return hidden

In [84]:
 net = Model()

In [70]:
for X, y in train_iter:
  
  h = net.init_hidden(len(X))
  out, h = net(X, h)
  print(out.shape,"\n")
  print(out,"\n", out.dtype)
  print(y, len(y))
  break

C:\Users\39388\AppData\Local\Temp\ipykernel_13376\112244331.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.sm(out)


torch.Size([64, 10]) 

tensor([[0.0901, 0.1151, 0.0808, 0.1046, 0.0859, 0.1169, 0.1171, 0.0837, 0.0979,
         0.1079],
        [0.0902, 0.1151, 0.0808, 0.1046, 0.0859, 0.1169, 0.1171, 0.0837, 0.0978,
         0.1079],
        [0.0902, 0.1151, 0.0808, 0.1045, 0.0859, 0.1170, 0.1171, 0.0837, 0.0979,
         0.1079],
        [0.0902, 0.1151, 0.0808, 0.1046, 0.0859, 0.1169, 0.1171, 0.0837, 0.0978,
         0.1078],
        [0.0901, 0.1151, 0.0808, 0.1044, 0.0858, 0.1170, 0.1171, 0.0838, 0.0979,
         0.1079],
        [0.0902, 0.1152, 0.0808, 0.1045, 0.0859, 0.1170, 0.1171, 0.0837, 0.0979,
         0.1079],
        [0.0902, 0.1152, 0.0808, 0.1046, 0.0859, 0.1169, 0.1171, 0.0837, 0.0978,
         0.1078],
        [0.0901, 0.1151, 0.0808, 0.1045, 0.0859, 0.1170, 0.1171, 0.0837, 0.0979,
         0.1079],
        [0.0901, 0.1151, 0.0808, 0.1044, 0.0858, 0.1170, 0.1171, 0.0838, 0.0980,
         0.1079],
        [0.0901, 0.1151, 0.0808, 0.1044, 0.0858, 0.1170, 0.1170, 0.0838, 0.0979,
     

In [71]:
for X, y in train_iter:
    print(y.type(torch.long))
    break

tensor([9, 8, 7, 2, 0, 3, 2, 0, 9, 1, 9, 8, 0, 0, 7, 2, 8, 1, 9, 1, 7, 9, 8, 1,
        9, 3, 4, 7, 2, 8, 7, 0, 7, 6, 3, 2, 5, 3, 6, 5, 1, 7, 4, 7, 6, 0, 3, 7,
        4, 5, 3, 2, 6, 2, 4, 6, 4, 9, 3, 3, 9, 0, 3, 7])


In [85]:
def train(train_loader, learn_rate, EPOCHS=5):
 
   
    # Defining loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=learn_rate)
    
    net.train()
    print("Starting Training of GRU model")

    epoch_times = []
    # Start training loop
    for epoch in range(1,EPOCHS+1):
        
        #start_time = d2l.Timer().start()
        
        h = net.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0
        
        for x, label in train_loader:
            counter += 1
            h = h.data
            net.zero_grad()
            out, h = net(x.to(device), h)
            label = label.type(torch.long)
            loss = criterion(out, label.to(device))
            loss.backward()
            optimizer.step()
            
    
            
            avg_loss += loss.item()
            if counter%200 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(
                    epoch, counter, len(train_loader), avg_loss/counter))
                
        
        
        
        #current_time = d2l.Timer().stop()

        print("Epoch {}/{} Done, Total Loss: {}".format(epoch, EPOCHS, avg_loss/len(train_loader)))
        #print("Total Time Elapsed: {} seconds".format(str(current_time-start_time)))
        #epoch_times.append(current_time-start_time)
    #print("Total Training Time: {} seconds".format(str(sum(epoch_times))))
    return net

In [86]:
learn_rate = 0.0001

#train(net, train_iter, test_iter, num_epochs, lr)
train(train_iter, learn_rate, EPOCHS=20)

Starting Training of GRU model


C:\Users\39388\AppData\Local\Temp\ipykernel_13376\112244331.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.sm(out)


RuntimeError: The size of tensor a (64) must match the size of tensor b (10) at non-singleton dimension 1